In [16]:
!pip install beautifulsoup4

   ---------------------------------------- 147.9/147.9 kB 8.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install pandas numpy scikit-learn nltk streamlit pillow requests

   ---------------------------------------- 8.7/8.7 MB 39.9 MB/s eta 0:00:00
   --------------------------------------- 658.1/658.1 kB 43.2 MB/s eta 0:00:00
   --------------------------------------- 207.3/207.3 kB 13.1 MB/s eta 0:00:00
   ---------------------------------------- 25.1/25.1 MB 38.4 MB/s eta 0:00:00
   ---------------------------------------- 6.9/6.9 MB 40.1 MB/s eta 0:00:00
   --------------------------------------- 241.6/241.6 kB 14.5 MB/s eta 0:00:00
   ---------------------------------------- 82.9/82.9 kB ? eta 0:00:00
   ---------------------------------------- 62.7/62.7 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 87.5/87.5 kB ? eta 0:00:00
   ---------------------------------------- 167.7/167.7 kB ? eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# load data
df = pd.read_csv('../data/rawdata.csv')

## Scrape images

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urlparse

def scrape_and_save_image(url, save_path):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        img_tag = soup.find('img', {'data-index': '0'})
        
        if img_tag and 'src' in img_tag.attrs:
            img_url = img_tag['src']
            img_response = requests.get(img_url, headers=headers)
            
            if img_response.status_code == 200:
                with open(save_path, 'wb') as f:
                    f.write(img_response.content)
                return True
    except Exception as e:
        print(f"Error scraping {url}: {str(e)}")
    return False

# Load the data
df = pd.read_csv('../data/rawdata.csv')

# Create images directory if it doesn't exist
os.makedirs('../data/images', exist_ok=True)

# Scrape and save images
for index, row in df.iterrows():
    product_id = urlparse(row['product_link']).path.split('/')[-1]
    image_path = f'../data/images/{product_id}.jpg'
    
    if not os.path.exists(image_path):
        success = scrape_and_save_image(row['product_link'], image_path)
        if success:
            print(f"Saved image for product {product_id}")
        else:
            print(f"Failed to save image for product {product_id}")

# Update DataFrame with local image paths
df['local_image_path'] = df['product_link'].apply(lambda x: f'../data/images/{urlparse(x).path.split("/")[-1]}.jpg')

# Save updated DataFrame
df.to_csv('../data/rawdata_with_images.csv', index=False)

print("Image scraping and CSV update complete.")

Failed to save image for product wooden-washer-and-dryer-top-laundry
Failed to save image for product the-complete-nursing-school-bundle-o
Failed to save image for product personalized-name-necklace-name-logo
Failed to save image for product marble-wood-charcuterie-board-wedding
Failed to save image for product madam-president-kamala-harris-and-tim
Failed to save image for product personalized-mama-sweatshirt-with-kid
Failed to save image for product personalized-hand-embroidered-name-baby
Failed to save image for product large-wood-name-sign-nursery-name-sign
Failed to save image for product personalized-name-necklace-by
Failed to save image for product personalized-signature-necklace-custom
Failed to save image for product wooden-world-map-enjoy-the-wood-home
Failed to save image for product x10-custom-wish-spell-make-a-wish-wish
Failed to save image for product custom-name-necklace-18k-gold-plated
Failed to save image for product ateez-mito-crochet-lightiny-cover
Failed to save imag

KeyboardInterrupt: 

# Data preprocessing

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
df = pd.read_csv('../data/rawdata.csv')

# Combine all tag columns into a single text feature
tag_columns = [f'tag_{i}' for i in range(1, 14)]  # Create a list of tag column names
df['text_features'] = df[tag_columns].fillna('').agg(' '.join, axis=1)

# Create TF-IDF vectors
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['text_features'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Function to get recommendations based on similarity
def get_recommendations(idx, cosine_sim=cosine_sim):
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:7]  # Get top 6 similar items
    product_indices = [i[0] for i in sim_scores]
    return df.iloc[product_indices]

# NLP Tag Matching

In [12]:
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')

def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().lower().replace('_', ' '))
    return list(synonyms)

def match_tags(recipient, occasion):
    recipient_synonyms = get_synonyms(recipient)
    occasion_synonyms = get_synonyms(occasion)
    
    matching_products = df[df['text_features'].apply(lambda x: any(tag in x.lower() for tag in recipient_synonyms + occasion_synonyms))]
    
    return matching_products

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\joshf\AppData\Roaming\nltk_data...


# Recommendation engine

In [13]:
def get_recommendations_by_category(matching_products):
    recommendations = {
        'Trending': matching_products.sort_values('listing_age', ascending=True).iloc[0],
        'Most Favorited': matching_products.sort_values('favorites', ascending=False).iloc[0],
        'Most Popular': matching_products.sort_values('est_mo_sales', ascending=False).iloc[0],
        'Quirky': matching_products.sample().iloc[0],
        'Best Seller': matching_products.sort_values('est_total_sales', ascending=False).iloc[0],
        'Highest Rated': matching_products.sort_values('avg_reviews', ascending=False).iloc[0]
    }
    return recommendations

def surprise_me(recipient, occasion):
    matching_products = match_tags(recipient, occasion)
    recommendations = get_recommendations_by_category(matching_products)
    return recommendations

In [14]:
import streamlit as st
import requests
from PIL import Image
from io import BytesIO

def load_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

st.title("Etsy Gift Finder")

recipient_options = ["Girlfriend", "Mom", "Dad", "Grandfather", "Grandmother", "Friend", "Coworker"]
occasion_options = ["Birthday", "Wedding", "Valentine's Day", "Christmas", "Anniversary", "Graduation"]

recipient = st.selectbox("Who is the gift for?", recipient_options)
occasion = st.selectbox("What's the occasion?", occasion_options)

if st.button("Surprise Me"):
    recommendations = surprise_me(recipient, occasion)
    
    for category, product in recommendations.items():
        st.subheader(category)
        st.write(product['product_name'])
        st.write(f"Price: ${product['price']}")
        st.write(f"Shop: {product['shop_name']}")
        
        image = load_image(product['product_link'])
        st.image(image, use_column_width=True)
        
        if st.button(f"View on Etsy", key=category):
            st.markdown(f"[View on Etsy]({product['product_link']})")
        
        st.write("---")

2024-09-20 09:33:30.420 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.011 
  command:

    streamlit run c:\Users\joshf\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-09-20 09:33:31.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.012 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.013 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-20 09:33:31.014 Session state does not function when running a script without `streamlit run`
2024-09-20 09:33:31.015 Thread 'MainThrea